# SEAIRT con effetto vaccinazione

Dal 27 dicembre 2020 (_vday_) è iniziata la somministrazione dei vaccini alla popolazione. Qui modifichiamo il modello SEAIRT per tenere conto di tale effetto.

In particolare, dal _vday_ si va a ridurre in modo costante il numero di individui suscettibili secondo un vaccination rate (_vr_) calcolato come il numero totale di seconde dosi somministrate finora diviso per il numero di giorni passati dal _vday_.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import time
import multiprocessing as mp
import pickle
from datetime import date
import matplotlib.dates as mdates
import requests
import json
import os
from datetime import date
from scipy.optimize import curve_fit
from scipy.integrate import solve_ivp,odeint
import scipy.stats as st
import math
from functools import partial
import urllib.request

In [2]:
data=pd.read_csv('https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale.csv')
data.set_index(pd.to_datetime(data.data).dt.date,inplace=True,drop=True)
data.drop(['data','stato','casi_da_sospetto_diagnostico','note',
           'tamponi','casi_testati','casi_da_screening',
           'variazione_totale_positivi','note_casi','note_test',
          'tamponi_test_molecolare','tamponi_test_antigenico_rapido'],axis=1,inplace=True)
print(data.columns)

Index(['ricoverati_con_sintomi', 'terapia_intensiva', 'totale_ospedalizzati',
       'isolamento_domiciliare', 'totale_positivi', 'nuovi_positivi',
       'dimessi_guariti', 'deceduti', 'totale_casi',
       'ingressi_terapia_intensiva', 'totale_positivi_test_molecolare',
       'totale_positivi_test_antigenico_rapido'],
      dtype='object')


In [3]:
#Data inspection
g=sns.relplot(data=data,height=6,aspect=1.2)

In [4]:
start=pd.to_datetime(data.index.min())
end=pd.to_datetime(data.index.max())
print('Periodo da: ',start,'\n a: ', end)
if (not (end-start).days==data.shape[0]-1) or data.index.duplicated(keep=False).sum()>0:
    print('Errore nei dati: giorni di troppo o giorni mancanti')

Periodo da:  2020-02-24 00:00:00 
 a:  2021-06-11 00:00:00


In [5]:
data_fit=data[['nuovi_positivi','totale_positivi','isolamento_domiciliare',
               'ricoverati_con_sintomi','terapia_intensiva']].copy()
data_fit['totale_casi']=data.nuovi_positivi.cumsum()
#print(data_fit.shape)
#data_fit

In [6]:
data_vax=pd.read_csv('https://raw.githubusercontent.com/italia/covid19-opendata-vaccini/master/dati/somministrazioni-vaccini-summary-latest.csv')
data_vax=data_vax.groupby('data_somministrazione')['seconda_dose'].sum()
print('Persone vaccinate con due dosi: ',sum(data_vax))
vr=sum(data_vax)/data_vax.shape[0]
print('Daily vaccination rate:',vr)

Persone vaccinate con due dosi:  12615405
Daily vaccination rate: 75541.34730538922


Oltre al numero di persone attualmente positive (I), teniamo traccia anche del numero cumulativo di infetti (In). Dalla differenza giornaliera di tale curva, possiamo poi ricavare la serie giornaliera dei nuovi infetti, da fittare sui dati.

In [7]:
# Funzione derivata da fornire a solve_ivp
def VSIR_ode(t,x,beta,alpha,eps,mui,gammar,gammat,pric,pt,vrate):
        S,E,A,I,In,C,R,T,V=x#,G,D=x
        
        dS=-beta*S*(I+A)/N-vrate
        dE=beta*S*(I+A)/N-eps*E
        dA=(1-alpha)*eps*E-mui*A
        dI=alpha*eps*E-mui*I
        dIn=alpha*eps*E
        dC=(1-pric)*mui*I-gammar*C
        dR=pric*(1-pt)*mui*I-gammar*R
        dT=pric*pt*mui*I-gammat*T
        dV=vrate
        return [dS,dE,dA,dI,dIn,dC,dR,dT,dV]

def VSIRinteg(x0p,ttotp,ti,beta,pric,pt):      
    sol=solve_ivp(VSIR_ode,[ttotp[0],ttotp[-1]],x0p,args=(beta,alpha,eps,mui,gammar,
                                                gammat,pric,pt,vrate),t_eval=ttotp)
    lung=len(sol.y[0])    
    return np.concatenate((np.diff(sol.y[4],prepend=x0p[4]).reshape(lung,1),
                           #sol.y[5].reshape(lung,1),
                           sol.y[6].reshape(lung,1),
                           sol.y[7].reshape(lung,1)),axis=1).flatten()


# Simulazione

Le condizioni iniziale sono prese dai dati, con due eccezioni.
La prima è il numero di suscettibili all'inizio, che viene impostato come la popolazione italiana tolti gli individui assegnati agli altri compartimenti.
La seconda è il numero di individui in incubazione all'inizio, che fissiamo a 0, per quanto verosimilmente maggiore.

Non utilizziamo per il fit i dati relativi agli ultimi 7 giorni, in modo da utilizzarli poi come test set per valutare l'accuratezza predittiva del modello.

In [8]:
#Durata predizioni
predlen=120
# Fitta anche curve senza tenere conto degli ultimi giorni
test_size=7
#Dopo quanto tempo si vedono gli effetti dei DPCM
delay=7


days_tot=len(pd.date_range(start,end))
tempo_tot=np.linspace(0,days_tot,days_tot+1)
N=60360000
x0=[N,data_fit.totale_positivi.iloc[0],data_fit.totale_positivi.iloc[0],data_fit.totale_positivi.iloc[0],
    0,data_fit.isolamento_domiciliare.iloc[0],
    data_fit.ricoverati_con_sintomi.iloc[0],data_fit.terapia_intensiva.iloc[0],0]
x0[0]=N-sum(x0[1:])
x0[4]=data_fit.totale_casi.iloc[0]

#Valori iniziali per la calibrazione:
beta=1/1.3
pric=0.5

#Parametri fissati:
#Frazione di sintomatici su totale infetti
alpha=0.15
#Tempo di incubazione
eps=1/2
#Tempo di permanenza in Infetti
mui=1/5.5
#Tempo di degenza in ospedale per chi non necessita terapia intensiva
gammar=1/25
#Tempo di degenza in terapia intensiva
gammat=1/6.5
#Frazione di ricoverati che necessita la terapia intensiva
pt=0.3
#Vaccination rate (diventa vr dal 27 dicembre)
vrate=0

dshift=(start-pd.to_datetime('2020-01-01')).days
vday=pd.to_datetime('2020-12-27')-pd.DateOffset(2) #inizio vaccinazione
#Giorni in cui cambiano le misure restrittive:
#(contati dall'inizio dell'anno: 1gennaio2020 = 0)
dvec=[0,70,82+delay,124+delay,138+delay,154+delay,166+delay,180+delay,251+delay,292+delay,304+delay,345+delay,
      365+38+delay,365+52+delay,365+74+delay,365+111+delay,
      days_tot+dshift-1]
dvec=[d-dshift for d in dvec]
dvec[0]=0
#print('dvec: ',dvec)

#Moltiplicatore di mobilità rispetto al periodo iniziale (da calibrare tramite beta)
mobs=[1,0.47,0.25,0.19,0.19,0.2,0.2,0.24,0.49,0.3,0.2,0.3,
     0.35,0.3,0.3,0.2]
betas=[beta*m for m in mobs]
#print('betas: ',betas)

popts=[]
pcovs=[]
integs=[]
for i,beta in enumerate(betas):
    dtotp=dvec[i+1]-dvec[i]
    ttotp=tempo_tot[dvec[i]:dvec[i+1]]
    ttotpred=tempo_tot[dvec[i]:dvec[i+1]+1]
    startp=data_fit.iloc[dvec[i]].name
    endp=data_fit.iloc[dvec[i+1]-1].name
    
    if startp>=vday:
        vrate=vr
    if i>2:
        pric=0.3
    if i==len(betas)-1:
        dtotp=dvec[i+1]-dvec[i]+1
        ttotp=tempo_tot[dvec[i]:dvec[i+1]+1]
        endp=data_fit.iloc[dvec[i+1]].name
        ttotpred=np.linspace(dvec[i],dvec[i+1]+predlen,dtotp+predlen+1)
    print('\nFitting from {} to {}'.format(startp,endp))
    x0p=x0
    if not i==0:
        x0p=[integ.y[j][-1] for j in range(len(integ.y))]
    #print('x0: ',x0p)
    
    fintegranda=partial(VSIRinteg,x0p,ttotp)
    popt, pcov = curve_fit(fintegranda, ttotp, 
                       np.array(data_fit.loc[startp:endp,['nuovi_positivi',
                                                          #'isolamento_domiciliare',
                                                          'ricoverati_con_sintomi',
                                                         'terapia_intensiva']].values).flatten(),
                       p0=[beta,pric,pt],maxfev=5000,bounds=([0.,0.,0.],
                                                              [2.5,1.,1.]))
    popts.append(popt)
    pcovs.append(pcov)
    integ=solve_ivp(VSIR_ode,[ttotpred[0],ttotpred[-1]],x0p,
                args=(popt[0],alpha,eps,mui,gammar,gammat,popt[1],popt[2],vrate),
                t_eval=ttotpred)
    integs.append(integ)
    #print('xfinal: ',[integ.y[j][-1] for j in range(len(integ.y))])
    #print('Ninf: ',integ.y[4][-1]-integ.y[4][-2])
    print('beta: ',popt[0])
    print('m (beta): ',popt[0]/popts[0][0])
    print('R0 (beta):',popt[0]/mui )
    print('prob intensiva:',popt[2] )
    print('prob ricovero:',popt[1] )


res=pd.DataFrame()
res['Susceptible']=np.concatenate([integ.y[0][:-1] for integ in integs])
res['Exposed']=np.concatenate([integ.y[1][:-1] for integ in integs])
res['Asymptomatic']=np.concatenate([integ.y[2][:-1] for integ in integs])
res['Infected']=np.concatenate([integ.y[3][:-1] for integ in integs])
res['ICumulative']=np.concatenate([integ.y[4][:-1] for integ in integs])
res['IsolamentoDomiciliare']=np.concatenate([integ.y[5][:-1] for integ in integs])
res['Ricoverati']=np.concatenate([integ.y[6][:-1] for integ in integs])
res['Terapia']=np.concatenate([integ.y[7][:-1] for integ in integs])
res['Vaccinati']=np.concatenate([integ.y[8][:-1] for integ in integs])
#I nuovi infetti giornalieri sono la differenza della curva cumulativa:
res['NInf']=np.diff(res['ICumulative'],prepend=0)
res['Day']=pd.date_range(start,end+pd.Timedelta(predlen,'D'))
res['Nday']=list(range(dvec[0]+dshift,dvec[-1]+dshift+predlen+1))
res['Rt']=np.concatenate([np.repeat(popts[i][0]/mui,len(integs[i].y[0][:-1])) 
                         for i in range(len(integs))])

res.set_index('Day',inplace=True)

print('\n betas fitted: ',[popt[0] for popt in popts])
print('\n m fitted: ',[popt[0]/popts[0][0] for popt in popts])
print('\n R0 fitted:',[popt[0]/mui for popt in popts])
print('\n prob intensiva fitted:',[popt[2] for popt in popts])
print('\n prob ricovero fitted:',[popt[1] for popt in popts])

#Diversi scenari con socialità al 110% e al 90% dell'attuale
integ=solve_ivp(VSIR_ode,[ttotpred[0],ttotpred[-1]],x0p,
                args=(popt[0]*1.1,alpha,eps,mui,gammar,gammat,popt[1],popt[2],vrate),
                t_eval=ttotpred)
res['WS_NuoviCasi']=np.nan
res.iloc[-len(integ.y[0])+1:,-1]=np.diff(integ.y[4])
res['WS_Ricoverati']=np.nan
res.iloc[-len(integ.y[0]):,-1]=integ.y[6]
res['WS_TerapiaIntensiva']=np.nan
res.iloc[-len(integ.y[0]):,-1]=integ.y[7]

integ=solve_ivp(VSIR_ode,[ttotpred[0],ttotpred[-1]],x0p,
                args=(popt[0]*.9,alpha,eps,mui,gammar,gammat,popt[1],popt[2],vrate),
                t_eval=ttotpred)
res['BS_NuoviCasi']=np.nan
res.iloc[-len(integ.y[0])+1:,-1]=np.diff(integ.y[4])
res['BS_Ricoverati']=np.nan
res.iloc[-len(integ.y[0]):,-1]=integ.y[6]
res['BS_TerapiaIntensiva']=np.nan
res.iloc[-len(integ.y[0]):,-1]=integ.y[7]

#Previsioni senza contare gli ultimi 7 giorni (test_size)
#Possibile solo se si è più di 7 giorni dopo il cambio di mobilità
if len(ttotp)>7:
    ttotp=tempo_tot[dvec[len(betas)-1]:dvec[len(betas)]+1-test_size]
    endp=data_fit.iloc[dvec[len(betas)]-test_size].name
    fintegranda=partial(VSIRinteg,x0p,ttotp)
    popt, pcov = curve_fit(fintegranda, ttotp, 
                           np.array(data_fit.loc[startp:endp,['nuovi_positivi',
                                                              #'isolamento_domiciliare',
                                                              'ricoverati_con_sintomi',
                                                             'terapia_intensiva']].values).flatten(),
                           p0=[beta,pric,pt],maxfev=5000,bounds=([0.,0.,0.],
                                                                  [2.5,1.,1.]))
    integ_test=solve_ivp(VSIR_ode,[ttotpred[0],ttotpred[-1]],x0p,
                    args=(popt[0],alpha,eps,mui,gammar,gammat,popt[1],popt[2],vrate),
                    t_eval=ttotpred)
    res['TestNInf']=np.nan
    res.iloc[-len(integ_test.y[0])+1:,-1]=np.diff(integ_test.y[4])
    res['TestRicoverati']=np.nan
    res.iloc[-len(integ_test.y[0]):,-1]=integ_test.y[6]
    res['TestTerapia']=np.nan
    res.iloc[-len(integ_test.y[0]):,-1]=integ_test.y[7]
    tested=True
else:
    print('\nNon è possibile fare testing su così pochi giorni dall\'ultimo cambio di mobilità')
    tested=False


Fitting from 2020-02-24 to 2020-03-10
beta:  0.8532898345726622
m (beta):  1.0
R0 (beta): 4.693094090149642
prob intensiva: 8.859589751267143e-15
prob ricovero: 0.9999999999999993

Fitting from 2020-03-11 to 2020-03-29
beta:  0.23384248330949986
m (beta):  0.27404812976192433
R0 (beta): 1.2861336582022491
prob intensiva: 0.23636148895899223
prob ricovero: 0.6929818173533331

Fitting from 2020-03-30 to 2020-05-10
beta:  0.13369022893676974
m (beta):  0.1566762236230359
R0 (beta): 0.7352962591522335
prob intensiva: 0.36633385716392936
prob ricovero: 0.30842142713414633

Fitting from 2020-05-11 to 2020-05-24
beta:  0.1417509556712473
m (beta):  0.1661228693088063
R0 (beta): 0.7796302561918601
prob intensiva: 0.9999999999997006
prob ricovero: 0.08151175018243667

Fitting from 2020-05-25 to 2020-06-09
beta:  0.1183034056317195
m (beta):  0.13864387086126165
R0 (beta): 0.6506687309744572
prob intensiva: 0.9999999999999999
prob ricovero: 0.09036958175233385

Fitting from 2020-06-10 to 2020-0

Nelle figure successive, la linea verticale arancione rappresenta il limite temporale dopo il quale non si sono considerati i dati per calibrare i parametri del modello. Tali dati sono quindi sconosciuti al modello e possono essere utilizzati per testarne l'effettiva capacità predittiva.

In [9]:
f,axs=plt.subplots(6,1,dpi=300,figsize=(8,16))
axs[0].plot(res.loc[start:end].index,data_fit.nuovi_positivi,'-o',alpha=.5,linewidth=1,c='grey',
           markersize=2)
axs[1].scatter(res.loc[start:end].index,data_fit.ricoverati_con_sintomi,alpha=.5,
               s=5,c='green')
axs[1].scatter(res.loc[start:end].index,data_fit.terapia_intensiva,alpha=.5,s=5,c='red')
axs[0].plot(res.index,res.NInf,'--',label='Pred Nuovi Sintomatici',linewidth=1)
axs[1].plot(res.index,res.Ricoverati,'--',label='PredRicoverati',c='green',linewidth=1)
axs[1].plot(res.index,res.Terapia,'--',label='PredTerapia',c='red',linewidth=1)

axs[2].plot(res.loc[start:end].index,data_fit.nuovi_positivi,'-o',alpha=.5,linewidth=1,c='grey',
           markersize=3)
axs[2].plot(res.index,res.NInf,'--',label='Pred Nuovi Sintomatici',linewidth=1)
axs[3].scatter(res.loc[start:end].index,data_fit.ricoverati_con_sintomi,alpha=.5,s=8,c='green')
axs[3].scatter(res.loc[start:end].index,data_fit.terapia_intensiva,alpha=.5,s=8,c='red')
axs[3].plot(res.index,res.Ricoverati,'--',label='PredRicoverati',c='green',linewidth=1)
axs[3].plot(res.index,res.Terapia,'--',label='PredTerapia',c='red',linewidth=1)
if tested:
    axs[2].plot(res.index,res.TestNInf,'--',label='TestNuoviCasi',c='darkblue',linewidth=1)
    axs[3].plot(res.index,res.TestRicoverati,'--',label='TestRicoverati',c='darkolivegreen',linewidth=1)
    axs[3].plot(res.index,res.TestTerapia,'--',label='TestTerapia',c='darkblue',linewidth=1)

axs[4].plot(res.loc[start:end].index,data_fit.nuovi_positivi,'-o',alpha=.5,linewidth=1,c='grey',
           markersize=2)
axs[4].plot(res.index,res.WS_NuoviCasi,'--',label='Worst_NuoviCasi',c='darkblue',linewidth=1)
axs[4].plot(res.index,res.BS_NuoviCasi,'--',label='Best_NuoviCasi',c='lightblue',linewidth=1)
axs[4].plot(res.index,res.NInf,'--',label='Pred Nuovi Sintomatici',linewidth=1)
axs[5].scatter(res.loc[start:end].index,data_fit.ricoverati_con_sintomi,alpha=.5,s=5,c='green')
axs[5].scatter(res.loc[start:end].index,data_fit.terapia_intensiva,alpha=.5,s=5,c='red')
axs[5].plot(res.index,res.WS_Ricoverati,'--',label='Worst_Ricoverati',c='darkolivegreen',linewidth=1)
axs[5].plot(res.index,res.BS_Ricoverati,'--',label='Best_Ricoverati',c='lightgreen',linewidth=1)
axs[5].plot(res.index,res.Ricoverati,'--',label='PredRicoverati',c='green',linewidth=1)
axs[5].plot(res.index,res.Terapia,'--',label='PredTerapia',c='red',linewidth=1)

axs[2].axvline(endp,linewidth=1,color='coral',label='Limite dati fittati')
axs[3].axvline(endp,linewidth=1,color='coral',label='Limite dati fittati')
for ax in axs:
    for dv in dvec[1:-1]:
        ax.axvline(res.index[dv], linewidth=1)
    ax.legend()
    plt.setp( ax.xaxis.get_majorticklabels(), rotation=70,fontsize=8 )



axs[0].set_ylim(-1,50000)
axs[1].set_ylim(-1,36000)

today=date.today()
axs[2].set_xlim(today-pd.Timedelta(14,unit='D'),today+pd.Timedelta(14,unit='D'))
axs[2].set_ylim(-1,70000)
#axs[1].set_xlim(dshift,dshift+30)
axs[3].set_xlim(today-pd.Timedelta(14,unit='D'),today+pd.Timedelta(14,unit='D'))
axs[3].set_ylim(-1,60000)


#axs[4].set_xlim('2020-02-27','2020-03-15')
#axs[4].set_ylim(-1,5000)
#axs[1].set_xlim(dshift,dshift+30)
#axs[5].set_xlim('2020-02-27','2020-03-15')
#axs[5].set_ylim(-1,7000)

axs[0].set_title('Totale dati')
axs[2].set_title('Previsioni 14gg')
axs[4].set_title('Previsioni 4 mesi (scenari multipli)')
#plt.legend()
plt.tight_layout()

### Esplorazione parametri

In [10]:
f,ax=plt.subplots(1,1,dpi=200)
plt.plot(res.index[7:-7],np.convolve(res['Rt'],np.ones(15,)/15,mode='valid'))
plt.xlim(start,end)
plt.axhline(1,linestyle='--',color='darkgrey')
plt.title('Rt')
plt.setp( ax.xaxis.get_majorticklabels(), rotation=70,fontsize=8 )
plt.show()